In [1]:
import time
import pydobot
from pydobot.dobot import MODE_PTP

In [3]:
ROBOT_PORT = 'COM8'
HOVER_DISTANCE = 85  # mm to elevate above pickup/drop locations

# Starting position
STARTING_POSITION = (295.1, 1.8, 152.3, 0.5)

# Safe transit point for movement between pallets
TRANSIT_POINT = (261.7, -27.8, 138.9, -5.2)

# Define source and target positions for 4 blocks
BLOCK_POSITIONS = [
    {'source': (263.2, -98.4, -14.7, -19.8),'target': (272.1, 18.6, -14.9, 4.2)},
    {'source': (210.8, -93.6, -14.3, -23.1), 'target': (220.4, 23.2, -11.1, 6.3)},
    {'source': (266.3, -35.4, -17.1, -19.4), 'target': (276.8, 79.3, -16.9, 4.1)},
    {'source': (209.7, -32.6, -13.9, -20.8),  'target': (219.5, 81.4, -14.1, 8.7)}
]

def execute_movement_with_hover(robot, position):
    # Create elevated position
    elevated_pos = (position[0], position[1], position[2] + HOVER_DISTANCE, position[3])
    robot.move_to(*elevated_pos, wait=True)
    # Descend to target position
    robot.move_to(*position, wait=True)
    time.sleep(0.6)

try:
    robot = Dobot(port=ROBOT_PORT)
    time.sleep(1.2)

    print("Navigating to starting position...")
    robot.move_to(*STARTING_POSITION, wait=True)

    # Phase 1: Transfer blocks from source to target locations
    for block_data in BLOCK_POSITIONS:
        print(f"Acquiring block from {block_data['source']}")
        execute_movement_with_hover(robot, block_data['source'])
        print("Engaging gripper...")
        robot.grip(True)
        time.sleep(0.6)

        # Navigate via transit point for safety
        print("Moving through transit point...")
        robot.move_to(*TRANSIT_POINT, wait=True)

        print(f"Placing block at {block_data['target']}")
        execute_movement_with_hover(robot, block_data['target'])
        print("Disengaging gripper...")
        robot.grip(False)
        time.sleep(0.6)

    # Phase 2: Return blocks to original positions in reverse sequence
    for block_data in reversed(BLOCK_POSITIONS):
        print(f"Retrieving block from {block_data['target']} for return")
        execute_movement_with_hover(robot, block_data['target'])
        print("Engaging gripper...")
        robot.grip(True)
        time.sleep(0.6)

        # Navigate via transit point for safety
        print("Moving through transit point...")
        robot.move_to(*TRANSIT_POINT, wait=True)

        print(f"Returning block to original location {block_data['source']}")
        execute_movement_with_hover(robot, block_data['source'])
        print("Disengaging gripper...")
        robot.grip(False)
        time.sleep(0.6)

    # Navigate back to starting position
    print("Returning to starting position...")
    robot.move_to(*STARTING_POSITION, wait=True)

except Exception as error_message:
    print("Error encountered:", error_message)

finally:
    if 'robot' in locals() and robot is not None:
        robot.close()
        print("Operation complete. Connection terminated.")

Error encountered: name 'Dobot' is not defined
